### Generating training data

In [1]:
import os
import gzip
import json
from random import randint
from bs4 import BeautifulSoup

In [2]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [5]:
dir_name = '/Users/aromanov/Documents/Corpus/HTML/NYT/2008/07'
files = []
for (dirpath, dirnames, filenames) in os.walk(dir_name):
    files += [os.path.join(dirpath, name) for name in filenames]

In [6]:
train = []

print('Total files =', len(files))
for i, file in enumerate(files[:]):
    print('\r', i+1, end='')
    if file.endswith('.html.gz'):
        with gzip.open(file) as gz:
            soup = BeautifulSoup(gz, 'lxml')
            text = ' '.join([p.text for p in soup.article.find_all('p', attrs={'class': 'story-content'})])
            text = text.replace('“', '"').replace('”', '"')
            doc = nlp(text)
            
            sentences = [sentence for sentence in doc.sents]
            i = 0
            while i < len(sentences):
                sentence = sentences[i]
                if len(sentence) > 2:
                    chunk = [[token.text, False] for token in sentence]
                    # glue every two sentences to get enough positive samples for training
                    if sentence[-1].text == '.' and i < len(sentences)-1:
                        chunk.pop()
                        chunk[-1][1] = True
                        chunk2 = [[token.text, False] for token in sentences[i+1]]
                        # 50% chance for lower case of next sent.
                        if randint(0,1): chunk2[0][0] = chunk2[0][0].lower()
                        chunk.extend(chunk2)
                        i += 1
                    train.append(chunk)
                i += 1

Total files = 6434
 6434

In [7]:
with gzip.open('run-on-train-sm.json.gz', 'wt+') as fw:
    json.dump(train, fw)